In [70]:
from semantic_search import SemanticSearch 
import pandas as pd
import openai
import praw
import os
import re

searcher = SemanticSearch()

pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', None)

In [71]:
REDDIT_CLIENT_ID = os.environ.get('REDDIT_CLIENT_ID')
REDDIT_CLIENT_SECRET = os.environ.get('REDDIT_CLIENT_SECRET')
REDDIT_USER_AGENT = os.environ.get('REDDIT_USER_AGENT')

reddit = praw.Reddit(client_id=REDDIT_CLIENT_ID, client_secret=REDDIT_CLIENT_SECRET, user_agent=REDDIT_USER_AGENT)

In [72]:
def generate_topics(query, model="gpt-3.5-turbo"):

    messages = [
        {"role": "user", "content": f"Take this query '{query}' and return a list of topics to input in Search so it returns good results. Each topic must stand on its own with respect to the relation of the question."},
    ]

    response = openai.ChatCompletion.create(
        model=model,
        messages=messages
    )

    response_message = response["choices"][0]["message"]["content"]

    topics = re.sub(r'^\d+\.\s*', '', response_message, flags=re.MULTILINE).split("\n")

    return topics

In [73]:
query = "Best pizza place in NYC"
topics = generate_topics(query)
print(topics)

['Top-rated pizza places in NYC', 'Best New York-style pizza restaurants', 'Highest reviewed pizzerias in New York City', 'Best pizza joints in Manhattan', 'Award-winning pizza spots in NYC', 'Famous pizza places in New York', 'Top 10 pizza places to try in NYC', 'Authentic pizza restaurants in New York', 'Best pizza slices in NYC', 'Gourmet pizza places in New York City']


In [74]:
posts = []

for topic in topics:
    for post in reddit.subreddit("all").search(
    topic, limit=200):
        posts.append([post.title, post.subreddit, post.selftext])

posts = pd.DataFrame(posts,columns=['title', 'subreddit', 'text'])

In [75]:
# Segments is title, text and subreddit at the end
segments = (posts['title'] + ' ' + posts['subreddit'].astype(str)).tolist()

In [76]:
searcher.fit(segments)

Fitting with n=5...


In [77]:
# TODO: Add distance check here
subreddits = set([result.split()[-1] for result in searcher(query)])

In [78]:
# Convert to string and "+" in between
subreddits = "+".join(subreddits)

In [79]:
final_posts = []

for post in reddit.subreddit(subreddits).search(
    query, limit=100):
        final_posts.append([post.title, post.subreddit, post.selftext])

final_posts = pd.DataFrame(final_posts,columns=['title', 'subreddit', 'text'])

In [80]:
final_segments = (final_posts['title'] + ' ' + final_posts['subreddit'].astype(str)).tolist()

In [81]:
final_searcher = SemanticSearch()

In [82]:
final_searcher.fit(final_segments)

Fitting with n=5...


In [83]:
final_searcher(query)

['Best Pizza in Manhattan? FoodNYC',
 'Best Pizza slice in NYC? FoodNYC',
 'Best Pizza, Manhattan NYC at 40.713050, -74.007230 The8BitRyanReddit',
 "Controversial: What's the best pizza place in all of NYC? newyorkcity",
 "Best pizza place in NYC that tourists DON'T know? circlejerknyc"]